In [1]:

!pip uninstall numpy
!pip install numpy==1.26.4

!pip install pmdarima




import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from tbats import TBATS
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import pmdarima as pm
import warnings

warnings.filterwarnings("ignore")

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which

In [2]:
# Load and preprocess data

import numpy as np
import pmdarima as pm
import pandas as pd

csv_file_path = '/content/Foreign_Exchange_Rates 2(in).csv'
df = pd.read_csv(csv_file_path)

df.columns = df.columns.str.strip()
df.drop(columns='Unnamed: 0', inplace=True)

df['Time Serie'] = pd.to_datetime(df['Time Serie'], errors='coerce', format='mixed', dayfirst=True)
df = df.dropna(subset=['Time Serie']) # Drop rows where Time Serie is NaT after coercion
df = df.sort_values('Time Serie')
df.set_index('Time Serie', inplace=True) # Explicitly set 'Time Serie' as index
df = df.apply(pd.to_numeric, errors='coerce').fillna(method='ffill')


df.fillna(method='ffill', inplace=True)  # forward fill

In [6]:
# Evaluate metrics

from sklearn.metrics import mean_absolute_error, mean_squared_error

def calculate_metrics(true, pred):
    mae = mean_absolute_error(true, pred)
    rmse = np.sqrt(mean_squared_error(true, pred))
    mape = np.mean(np.abs((true - pred) / true)) * 100
    return mae, rmse, mape

# Forecast wrappers
def arima_forecast(train, h):
    model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    return model.predict(n_periods=h)

def ets_forecast(train, h):
    model = ExponentialSmoothing(train, trend='add', seasonal=None).fit()
    return model.forecast(h)

def prophet_forecast(train, h):
    df_p = pd.DataFrame({'ds': train.index, 'y': train.values})
    model = Prophet(daily_seasonality=False, yearly_seasonality=True)
    model.fit(df_p)
    future = model.make_future_dataframe(periods=h)
    forecast = model.predict(future)
    return forecast['yhat'].iloc[-h:].values

def tbats_forecast(train, h):
    estimator = TBATS(seasonal_periods=[7, 30, 365])
    model = estimator.fit(train)
    return model.forecast(steps=h)

def lstm_forecast(train, h, n_lag=30):
    values = train.values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)

    X, y = [], []
    for i in range(n_lag, len(scaled) - h):
        X.append(scaled[i - n_lag:i])
        y.append(scaled[i:i + h].flatten())

    if len(X) == 0:
        raise ValueError("Not enough data for LSTM.")

    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    model = Sequential([
        LSTM(50, activation='relu', input_shape=(n_lag, 1)),
        Dense(h)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=10, verbose=0)

    last_seq = scaled[-n_lag:].reshape((1, n_lag, 1))
    prediction = model.predict(last_seq)
    return scaler.inverse_transform(prediction).flatten()

# Main evaluation
results = []
forecast_horizon = 60

for col in df.columns:
    print(f"Evaluating {col}...")
    series = df[col]
    train, test = series[:-forecast_horizon], series[-forecast_horizon:]

    models = {
        "ARIMA": arima_forecast,
        "ETS": ets_forecast,
        "Prophet": prophet_forecast,
        "TBATS": tbats_forecast,
        "LSTM": lstm_forecast
    }

    for model_name, model_func in models.items():
        try:
            preds = model_func(train, forecast_horizon)
            mae, rmse, mape = calculate_metrics(test.values, preds)
            results.append({
                "Currency": col,
                "Model": model_name,
                "MAE": mae,
                "RMSE": rmse,
                "MAPE": mape
            })
        except Exception as e:
            print(f"⚠️ {model_name} failed on {col}: {e}")
            results.append({
                "Currency": col,
                "Model": model_name,
                "MAE": np.nan,
                "RMSE": np.nan,
                "MAPE": np.nan
            })

# Save all results
results_df = pd.DataFrame(results)
results_df.to_csv("model_evaluation_all_models.csv", index=False)

# Identify best model per currency (by RMSE)
best_models = results_df.loc[results_df.groupby('Currency')['RMSE'].idxmin()]
best_models.to_csv("best_model_per_currency.csv", index=False)

print("\n✅ All done!")
print("• Full metrics: model_evaluation_all_models.csv")
print("• Best models: best_model_per_currency.csv")

Evaluating AUSTRALIA - AUSTRALIAN DOLLAR/US$...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/uo720y1j.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/xh0t_90k.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83866', 'data', 'file=/tmp/tmp88qbtwgk/uo720y1j.json', 'init=/tmp/tmp88qbtwgk/xh0t_90k.json', 'output', 'file=/tmp/tmp88qbtwgk/prophet_modell639br5c/prophet_model-20250716180524.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:05:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:05:27 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Evaluating EURO AREA - EURO/US$...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/bb8mmc8y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/wwokc7zf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56511', 'data', 'file=/tmp/tmp88qbtwgk/bb8mmc8y.json', 'init=/tmp/tmp88qbtwgk/wwokc7zf.json', 'output', 'file=/tmp/tmp88qbtwgk/prophet_modelrisaunmv/prophet_model-20250716182636.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:26:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:26:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Evaluating NEW ZEALAND - NEW ZELAND DOLLAR/US$...


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Evaluating BRAZIL - REAL/US$...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/w6hcechs.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/jp49psht.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=68986', 'data', 'file=/tmp/tmp88qbtwgk/w6hcechs.json', 'init=/tmp/tmp88qbtwgk/jp49psht.json', 'output', 'file=/tmp/tmp88qbtwgk/prophet_modelmdzbnwmq/prophet_model-20250716192423.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:24:23 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:24:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
Evaluating CANADA - CANADIAN DOLLAR/US$...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/c2khm2jd.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp88qbtwgk/srm2auml.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=2860', 'data', 'file=/tmp/tmp88qbtwgk/c2khm2jd.json', 'init=/tmp/tmp88qbtwgk/srm2auml.json', 'output', 'file=/tmp/tmp88qbtwgk/prophet_model5ue0de5h/prophet_model-20250716195721.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:57:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:57:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


KeyboardInterrupt: 